In [0]:
pip install web3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.5/587.5 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 109.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.5/797.5 kB 40.1 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


**## Batch processing of Ethereum data from AWS S3**

In [0]:
# Databricks notebook source
# === SIMPLE PARAMETERIZATION (ONLY NECESSARY VARIABLES) ===
# Parameterize only what needs to be variable for reusability
dbutils.widgets.text("catalog_name", "blockchain", "Catalog Name")
dbutils.widgets.text("schema_name", "ethereum", "Schema Name")
dbutils.widgets.text("num_files", "20", "Number of Files to Download")

# === CONFIGURATION ===
# Get widget values
CATALOG = dbutils.widgets.get("catalog_name")
SCHEMA = dbutils.widgets.get("schema_name")
NUM_FILES = int(dbutils.widgets.get("num_files"))

# Hard-coded values (no need to parameterize constants)
AWS_BUCKET = "aws-public-blockchain"
S3_PREFIX = "v1.0/eth/blocks/"

# Unity Catalog volume paths for data organization
DATA_VOLUME = f"/Volumes/{CATALOG}/{SCHEMA}/ethereum"
CHECKPOINT_VOLUME = f"/Volumes/{CATALOG}/{SCHEMA}/ethereum_checkpoints"
SCHEMA_VOLUME = f"/Volumes/{CATALOG}/{SCHEMA}/ethereum_schemas"

print(f"🔧 Using Catalog: {CATALOG}, Schema: {SCHEMA}")
print(f"📦 Downloading {NUM_FILES} files from s3://{AWS_BUCKET}/{S3_PREFIX}")

# === UNITY CATALOG SETUP ===
stmts = [
    f"CREATE CATALOG IF NOT EXISTS {CATALOG}",
    f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{SCHEMA}",
    f"CREATE VOLUME IF NOT EXISTS {CATALOG}.{SCHEMA}.ethereum",
    f"CREATE VOLUME IF NOT EXISTS {CATALOG}.{SCHEMA}.ethereum_checkpoints",
    f"CREATE VOLUME IF NOT EXISTS {CATALOG}.{SCHEMA}.ethereum_schemas",
]

for i, s in enumerate(stmts, 1):
    print(f"[{i}/{len(stmts)}] {s}")
    try:
        spark.sql(s)
        print("  ✅ Success")
    except Exception as e:
        print(f"  ❌ Error: {e}")

print(f"\nCreated/verified UC objects. Paths available:")
print(f"  Data: {DATA_VOLUME}")
print(f"  Checkpoints: {CHECKPOINT_VOLUME}")
print(f"  Schemas: {SCHEMA_VOLUME}")

# === DATA DOWNLOAD ===
import os
import boto3
from botocore import UNSIGNED
from botocore.client import Config

print(f"\n📥 Downloading to: {DATA_VOLUME}")
os.makedirs(DATA_VOLUME, exist_ok=True)

# Configure anonymous S3 client (no AWS credentials needed!)
s3 = boto3.client("s3", config=Config(signature_version=UNSIGNED))

# Collect parquet files from S3
keys = []
token = None

while len(keys) < NUM_FILES:
    params = {
        "Bucket": AWS_BUCKET, 
        "Prefix": S3_PREFIX, 
        "MaxKeys": min(1000, NUM_FILES - len(keys))
    }
    if token:
        params["ContinuationToken"] = token
    
    resp = s3.list_objects_v2(**params)
    
    for obj in resp.get("Contents", []) or []:
        if obj["Key"].endswith(".parquet"):
            keys.append(obj["Key"])
            if len(keys) >= NUM_FILES:
                break
    
    if not resp.get("IsTruncated"):
        break
    token = resp.get("NextContinuationToken")

if not keys:
    raise RuntimeError(f"No parquet files found under s3://{AWS_BUCKET}/{S3_PREFIX}")

# Download files with progress tracking
for i, key in enumerate(keys, 1):
    rel_path = key.replace("v1.0/eth/", "")
    dest_path = os.path.join(DATA_VOLUME, rel_path)
    os.makedirs(os.path.dirname(dest_path), exist_ok=True)
    
    print(f"[{i}/{len(keys)}] {os.path.basename(key)} ...", end=" ", flush=True)
    s3.download_file(AWS_BUCKET, key, dest_path)
    print("✓")

print("✅ Download complete!")

🔧 Using Catalog: blockchain, Schema: ethereum
📦 Downloading 20 files from s3://aws-public-blockchain/v1.0/eth/blocks/
[1/5] CREATE CATALOG IF NOT EXISTS blockchain
  ✅ Success
[2/5] CREATE SCHEMA IF NOT EXISTS blockchain.ethereum
  ✅ Success
[3/5] CREATE VOLUME IF NOT EXISTS blockchain.ethereum.ethereum
  ✅ Success
[4/5] CREATE VOLUME IF NOT EXISTS blockchain.ethereum.ethereum_checkpoints
  ✅ Success
[5/5] CREATE VOLUME IF NOT EXISTS blockchain.ethereum.ethereum_schemas
  ✅ Success

Created/verified UC objects. Paths available:
  Data: /Volumes/blockchain/ethereum/ethereum
  Checkpoints: /Volumes/blockchain/ethereum/ethereum_checkpoints
  Schemas: /Volumes/blockchain/ethereum/ethereum_schemas

📥 Downloading to: /Volumes/blockchain/ethereum/ethereum
[1/20] part-00000-32767f69-9150-49ac-9c03-45f34b103c34-c000.snappy.parquet ... ✓
[2/20] part-00000-62c9c86c-8a10-4196-b54c-01a2a139f4ec-c000.snappy.parquet ... ✓
[3/20] part-00000-5438c668-b9c9-4b0a-8a35-64ff30b73cdf-c000.snappy.parquet ... 

In [0]:
CATALOG = dbutils.widgets.get("catalog_name")
SCHEMA = dbutils.widgets.get("schema_name")
NUM_FILES = int(dbutils.widgets.get("num_files"))


# Unity Catalog volume paths for data organization
DATA_VOLUME = f"/Volumes/{CATALOG}/{SCHEMA}/ethereum"
CHECKPOINT_VOLUME = f"/Volumes/{CATALOG}/{SCHEMA}/ethereum_checkpoints"
SCHEMA_VOLUME = f"/Volumes/{CATALOG}/{SCHEMA}/ethereum_schemas"

DATA_VOLUME

'/Volumes/blockchain/ethereum/ethereum'

In [0]:
import os
import time
import logging
import json
import pandas as pd
from typing import Any, Dict
from web3 import Web3
from web3.datastructures import AttributeDict
from hexbytes import HexBytes
from requests.exceptions import HTTPError, ConnectionError, Timeout
from web3.exceptions import Web3Exception

# ---------------------- Logging ----------------------
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# ---------------------- Rate Limiter ----------------------
class RateLimiter:
    """Simple rate limiter to control API call frequency"""
    def __init__(self, max_calls_per_second: float = 5.0):
        self.max_calls_per_second = max_calls_per_second
        self.min_interval = 1.0 / max_calls_per_second
        self.last_call_time = 0
    
    def wait_if_needed(self):
        current_time = time.time()
        time_since_last_call = current_time - self.last_call_time
        if time_since_last_call < self.min_interval:
            time.sleep(self.min_interval - time_since_last_call)
        self.last_call_time = time.time()

# ---------------------- Utilities ----------------------
def check_api_response_status(w3: Web3) -> bool:
    """Check if Web3 API endpoint is responding"""
    try:
        block_number = w3.eth.block_number
        if block_number > 0:
            logger.info(f"✅ API responding correctly (current block: {block_number})")
            return True
        else:
            logger.error(f"❌ Invalid block number: {block_number}")
            return False
    except Exception as e:
        logger.error(f"❌ API connection error: {e}")
        return False

def to_serializable(obj: Any) -> Any:
    """Convert Web3 objects to JSON/Parquet-safe Python objects"""
    if isinstance(obj, (AttributeDict, dict)):
        return {k: to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [to_serializable(x) for x in obj]
    elif isinstance(obj, HexBytes):
        return obj.hex()
    elif isinstance(obj, bytes):
        return obj.hex()
    elif isinstance(obj, int):
        if abs(obj) > 2**63-1:
            return str(obj)
        return obj
    elif hasattr(obj, '__dict__'):
        return to_serializable(obj.__dict__)
    else:
        return obj

def make_block_parquet_safe(block_dict: Dict[str, Any]) -> Dict[str, Any]:
    """Convert block dict to Parquet-safe format"""
    safe_block = {}
    for k, v in block_dict.items():
        if isinstance(v, int) and abs(v) > 2**63 - 1:
            safe_block[k] = str(v)
        elif k == "timestamp":
            safe_block[k] = v
            safe_block["timestamp_dt"] = pd.to_datetime(v, unit="s")
        elif k == "transactions":
            # Convert all transactions to JSON string, converting big ints inside
            safe_block[k] = json.dumps([
                {tk: str(tv) if isinstance(tv, int) and abs(tv) > 2**63-1 else tv for tk, tv in tx.items()}
                for tx in v
            ])
        else:
            safe_block[k] = v
    return safe_block

def fetch_block_with_retry(w3: Web3, block_number: int, rate_limiter: RateLimiter,
                           max_retries: int = 3, base_delay: float = 1.0) -> Dict[str, Any]:
    """Fetch a block with retry and rate limiting"""
    for attempt in range(max_retries):
        try:
            rate_limiter.wait_if_needed()
            block = w3.eth.get_block(block_number, full_transactions=True)
            return to_serializable(block)
        except (HTTPError, ConnectionError, Timeout) as e:
            if attempt == max_retries - 1:
                logger.error(f"Failed to fetch block {block_number} after {max_retries} attempts: {e}")
                raise
            delay = base_delay * (2 ** attempt)
            logger.warning(f"Attempt {attempt + 1} failed for block {block_number}, retrying in {delay}s: {e}")
            time.sleep(delay)
        except Exception as e:
            logger.error(f"Unexpected error fetching block {block_number}: {e}")
            raise

# ---------------------- Main Loop ----------------------
def main():
    # ---------- Configuration ----------
    provider_uri = "https://mainnet.infura.io/v3/31966bfed8674afaab1a9d9ba201665a"
    output_dir = f"{DATA_VOLUME}/blocks"
   # os.makedirs(output_dir, exist_ok=True)
    rate_limiter = RateLimiter(max_calls_per_second=0.8)  # safe for Infura free tier

    # ---------- Initialize Web3 ----------
    w3 = Web3(Web3.HTTPProvider(provider_uri))
    if not w3.is_connected():
        logger.error("❌ Failed to connect to Ethereum node")
        return
    logger.info("Connected to Ethereum mainnet")

    # ---------- Start from latest block ----------
    last_block = w3.eth.block_number
    logger.info(f"Starting from block {last_block}")

    # ---------- Loop for new blocks ----------
    while True:
        try:
            rate_limiter.wait_if_needed()
            latest_block = w3.eth.block_number
            blocks_to_process = list(range(last_block + 1, latest_block + 1))

            if blocks_to_process:
                logger.info(f"Processing blocks {last_block + 1} to {latest_block}")

            for block_number in blocks_to_process:
                try:
                    logger.debug(f"Processing block {block_number}")
                    block_dict = fetch_block_with_retry(w3, block_number, rate_limiter)
                    block_safe = make_block_parquet_safe(block_dict)

                    # Save block to Parquet
                    block_file = os.path.join(output_dir, f"block_{block_number}.parquet")
                    pd.DataFrame([block_safe]).to_parquet(block_file, engine="pyarrow", index=False)

                    # Save transactions separately (JSON string → optional to decode later)
                    tx_file = os.path.join(output_dir, f"transactions_block_{block_number}.parquet")
                    transactions = json.loads(block_safe["transactions"])
                    if transactions:
                        pd.DataFrame(transactions).to_parquet(tx_file, engine="pyarrow", index=False)
                        tx_count = len(transactions)
                    else:
                        tx_count = 0

                    logger.info(f"✅ Saved block {block_number} with {tx_count} transactions")

                except Exception as e:
                    logger.error(f"❌ Failed to process block {block_number}: {e}")
                    if "web3" in str(e).lower() or "connection" in str(e).lower():
                        check_api_response_status(w3)
                    continue

            last_block = latest_block
            time.sleep(15)  # wait for ~1 block (~12-13s)

        except KeyboardInterrupt:
            logger.info("Stopping block fetcher...")
            break
        except Exception as e:
            logger.error(f"Unexpected error in main loop: {e}")
            time.sleep(30)

if __name__ == "__main__":
    main()


2025-09-23 03:41:32,204 - INFO - Connected to Ethereum mainnet
2025-09-23 03:41:32,231 - INFO - Starting from block 23423109
2025-09-23 03:41:47,277 - INFO - Processing blocks 23423110 to 23423110
2025-09-23 03:41:49,194 - INFO - ✅ Saved block 23423110 with 235 transactions
2025-09-23 03:42:04,219 - INFO - Processing blocks 23423111 to 23423112
2025-09-23 03:42:05,721 - ERROR - ❌ Failed to process block 23423111: ("Could not convert '79999850508603262000' with type str: tried to convert to int64", 'Conversion failed for column value with type object')
2025-09-23 03:42:06,970 - ERROR - ❌ Failed to process block 23423112: ("Could not convert '13263930644083825000' with type str: tried to convert to int64", 'Conversion failed for column value with type object')
2025-09-23 03:42:21,992 - INFO - Processing blocks 23423113 to 23423113
2025-09-23 03:42:23,534 - ERROR - ❌ Failed to process block 23423113: ("Could not convert '40569500000000000000' with type str: tried to convert to int64", 'Co

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
df = spark.read.parquet("/Volumes/blockchain/ethereum/ethereum/blocks/block_23422923.parquet")

display(df)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5916442043934546>, line 3
      1 df = spark.read.parquet("/Volumes/blockchain/ethereum/ethereum/blocks/block_23422923.parquet")
----> 3 display(df)

File /databricks/python_shell/lib/dbruntime/display.py:133, in Display.display(self, input, *args, **kwargs)
    131     pass
    132 elif self._cf_helper is not None and isinstance(input, ConnectDataFrame):
--> 133     self.display_connect_table(input, **kwargs)
    134 elif isinstance(input, ConnectDataFrame):
    135     if input.isStreaming:

File /databricks/python_shell/lib/dbruntime/display.py:93, in Display.display_connect_table(self, df, **kwargs)
     88 except Exception as e:
     89     raise type(
     90         e
     91     )("IPython shell encountered an error or was missing data, please restart the notebook or contact Databricks support"
     92       ) from

In [0]:
reader = (
    spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", "parquet")  
        .option("cloudFiles.schemaLocation", SCHEMA_VOLUME)  
        .option("checkpointLocation", CHECKPOINT_VOLUME)  # Required
        .option("cloudFiles.schemaEvolutionMode", "addNewColumns")  
        .option("cloudFiles.schemaHints", "number BIGINT, baseFeePerGas BIGINT")  
        .load(f"dbfs:{DATA_VOLUME}/blocks/")
)

# Use append mode for non-aggregated streaming
#display(reader, checkpointLocation=CHECKPOINT_VOLUME, outputMode="append")

# Extract and transform block-level fields if needed
blocks_df = reader.select(
   "*",
   "_metadata"
)

# Write to Delta using Structured Streaming
blocks_query = (
    blocks_df.writeStream
        .format("delta")  # Delta Lake sink for ACID transactions
        .outputMode("append")  # Append new blocks as they arrive
        .option("checkpointLocation", f"{CHECKPOINT_VOLUME}/blocks/")  # State management
        .trigger(availableNow=True)  # Process all available data
        .table(f"{CATALOG}.{SCHEMA}.blocks")  # Save as managed Delta table
)


2025-09-23 03:11:55,280 - INFO - Received command c on object id p0


In [0]:
%sql
-- Total blocks ingested
SELECT COUNT(*) 
FROM blockchain.ethereum.blocks;



COUNT(*)
0


In [0]:
%sql

-- Total transactions ingested
SELECT COUNT(*) 
FROM ethereum.transactions;



COUNT(*)
2441


In [0]:
%sql
-- Top miners by number of blocks mined
SELECT miner, COUNT(*) AS blocks_mined
FROM ethereum.blocks
GROUP BY miner
ORDER BY blocks_mined DESC;


miner,blocks_mined
0xdadB0d80178819F2319190D340ce9A924f783711,8
0x4838B106FCe9647Bdf1E7877BF73cE8B0BAD5f97,2
0x388C818CA8B9251b393131C08a736A67ccB19297,1


In [0]:
%sql

-- High-value transfers (greater than 1 ETH)
SELECT *
FROM ethereum.transactions
WHERE CAST(value AS DECIMAL(38,0)) > 1000000000000000000;

block_number,tx_hash,from_address,to_address,value,nonce,gas
23415045,4c760d62865a16248c300067ad0286415b740899f28e97ffad9b749efbe0d1d0,0xf4da41ebED948663f988aFaf80c53996b9a6834d,0x1231DEB6f5749EF6cE6943a275A1D3E7486F4EaE,1160000000000000000,139,177350
23415045,6177a3046243563c8f79fe7e99b8ca61f76199716e87c4a682528a9049839983,0x95793142648358E4De193Da6507EBA28e4A7cDFf,0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D,1700000000000000000,7,300000
23415045,7cd0c3ddde44f8a40ad009665894673e67631cc68c2365c70296d01db067a469,0x21a31Ee1afC51d94C2eFcCAa2092aD1028285549,0xcF30FcF35845713a7A86915b6DbfA27908286eD0,1961316000000000000,12956996,207128
23415046,b1a52c05fd7fc25a0616ee76d3ac3becca67dbed5cbfad0848637297d952331c,0xf35eaA2F01CDbc11C5181751528970F95BFea253,0xa03400E098F4421b34a3a44A1B4e571419517687,50516796971647967000,1810,30000
23415046,6dbef7f8cc6b4338e040b5afad94eec0692b2a6683e27dc97e9e92c30517919c,0x41eb5aB31Aa0e4A4027547C05853af2Fd13A11Dc,0x241f4848678302Fa027DFdF05f6d548fe3CDeaf7,183180000000000000000,11,24150
23415046,b2255f7aad15f306316d752d9a68c5cb3571d75c7cb58b5efaf40cfdd9837678,0xaF62aecC38802DE1765733C40c55a28584779729,0xaF62aecC38802DE1765733C40c55a28584779729,1359691118269870569,143,94041
23415046,389844477f2dddeae662691efab04b5a8e8568a7a9c83b138e56cece4e03b916,0xdD9BD62D1c08210fDDa6f841eD5196B45A939625,0xd01607c3C5eCABa394D8be377a08590149325722,222000000000000000000,2773,300000
23415046,dca652a1502ef80b5b7c2955212f1fb9116107bfaf353573e232b0290fb62576,0xaEAb0c0499cED7cB0cf32A15d642530B3c96bbE3,0xA9D1e08C7793af67e9d92fe308d5697FB81d3E43,499999953400000000000,0,23300
23415046,ca26ffaeca384e82cd883f97a02a32a9266019cbd575b56956b59e8c42506845,0x4BE9b904707c0E346f9629e2e29F71bf87F49dD1,0x5FE6cF1542C27a2a19bB28f427d546F296cdfa25,2000000000000000000,17,21000
23415046,a75a8ce3d06759b6b430d4ba07873f01b8328eb7a2c8feb8399826a1693363c5,0x9696f59E4d72E237BE84fFD425DCaD154Bf96976,0xF214798A8aF12Ad98e173171ee2D8d7ea11CE75C,24836000000000000000,8231593,207128
